In [ ]:
!pip install -q \
  langchain==0.1.16 \
  langchain-community==0.0.38 \
  langchain-openai==0.1.6 \
  tavily-python \
  requests==2.32.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires re

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

from google.colab import userdata
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

Importing openai and tavily api keys

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
search_tool = TavilySearchResults(k=5)

def get_weather(city):
  return requests.get(f"https://wttr.in/{city}?format=3").text

weather_tool = Tool(
    name="WeatherTool",
    func=get_weather,
    description="Getting weather"
)
tools = [search_tool,weather_tool]



We are using gpt 4o mini due to it's low cost and high speed. And we are using tavily search with top 5 search results. Weather tool to help the model get the accurate weather details for a location.

In [ ]:
system_prompt = """
You are an AI travel planning agent.

MANDATORY RULES (failure = regenerate):
1. Every itinerary MUST include a dedicated section called "Accommodation Options".
2. This section MUST list 2–3 REAL hotel names.
3. Each hotel MUST include:
   - Hotel name
   - Area/locality
   - Price per night in INR
   - Booking source (Booking.com, MakeMyTrip, Goibibo, etc.)
4. Hotel prices MUST scale with the total trip budget.
5. You MUST use web search to find hotels. Do not invent names.
6. If hotel names are missing, the answer is INVALID.

Hotel budget tiers:
- Total budget < ₹30,000 → ₹1,500–₹3,000/night
- ₹30,000–₹70,000 → ₹3,000–₹6,000/night
- ₹70,000–₹1,20,000 → ₹6,000–₹10,000/night
- > ₹1,20,000 → ₹10,000–₹18,000/night

OUTPUT FORMAT (STRICT):

Accommodation Options:
- Hotel Name – Area – ₹price/night (source)

Then provide day-wise itinerary and cost breakdown.
"""



Initial prompt to the LLM.

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,
    verbose=True,
    system_message = system_prompt
)

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
agent.run("""
Plan a 5-day trip to Delhi from Chennai under ₹150000 from 4th Feb. It needs to be luxurious with good food and transport in Delhi via cab.
""")

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Here's a luxurious 5-day trip plan to Delhi from Chennai, focusing on comfort, good food, and convenient transportation. The budget is set at ₹150,000.

### Day 1: Arrival in Delhi
- **Flight from Chennai to Delhi**: Book a round-trip flight. A business class ticket can cost around ₹15,000 - ₹20,000.
- **Check-in**: Stay at a luxury hotel like The Oberoi or The Taj Mahal Hotel. Average cost per night: ₹15,000.
- **Lunch**: Enjoy a meal at the hotel or a nearby fine dining restaurant (approx. ₹2,000).
- **Evening Activity**: Visit India Gate and take a leisurely walk. 
- **Dinner**: Dine at a renowned restaurant like Bukhara or Dum Pukht (approx. ₹3,000).
- **Total for Day 1**: ₹20,000 (flight) + ₹15,000 (hotel) + ₹2,000 (lunch) + ₹3,000 (dinner) = ₹40,000.

### Day 2: Explore Delhi
- **Breakfast**: At the hotel (included in stay).
- **Morning Activity**: Visit Qutub Minar and Humayun's Tomb (entry fees approx. ₹500).
- **Lunch**: At a luxury rest

"Here's a luxurious 5-day trip plan to Delhi from Chennai, focusing on comfort, good food, and convenient transportation. The budget is set at ₹150,000.\n\n### Day 1: Arrival in Delhi\n- **Flight from Chennai to Delhi**: Book a round-trip flight. A business class ticket can cost around ₹15,000 - ₹20,000.\n- **Check-in**: Stay at a luxury hotel like The Oberoi or The Taj Mahal Hotel. Average cost per night: ₹15,000.\n- **Lunch**: Enjoy a meal at the hotel or a nearby fine dining restaurant (approx. ₹2,000).\n- **Evening Activity**: Visit India Gate and take a leisurely walk. \n- **Dinner**: Dine at a renowned restaurant like Bukhara or Dum Pukht (approx. ₹3,000).\n- **Total for Day 1**: ₹20,000 (flight) + ₹15,000 (hotel) + ₹2,000 (lunch) + ₹3,000 (dinner) = ₹40,000.\n\n### Day 2: Explore Delhi\n- **Breakfast**: At the hotel (included in stay).\n- **Morning Activity**: Visit Qutub Minar and Humayun's Tomb (entry fees approx. ₹500).\n- **Lunch**: At a luxury restaurant like The Spice Rout